<a href="https://colab.research.google.com/github/styxx216/ML/blob/main/rec_sis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightfm

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from lightfm import LightFM

#model selection
import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense , merge



from keras.layers.merge import dot
from keras.models import Model


# specifically for deeplearning.
from keras.layers import Dropout, Flatten,Activation,Input,Embedding
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
from IPython.display import SVG

from tensorflow.keras.optimizers import Adam

In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [ ]:
PATH_GENOME_SCORES = '/content/drive/MyDrive/datasets/MovieLens/genome_scores.csv'
PATH_GENOME_TAGS = '/content/drive/MyDrive/datasets/MovieLens/genome_tags.csv'
PATH_LINK = '/content/drive/MyDrive/datasets/MovieLens/link.csv'
PATH_MOVIE = '/content/drive/MyDrive/datasets/MovieLens/movie.csv'
PATH_RATING = '/content/drive/MyDrive/datasets/MovieLens/rating.csv'
PATH_TAG = '/content/drive/MyDrive/datasets/MovieLens/tag.csv'

In [ ]:
train_test_split = 0.2

In [ ]:
genome_scores = pd.read_csv(PATH_GENOME_SCORES)
tags_df = pd.read_csv(PATH_TAG)
genome_tags = pd.read_csv(PATH_GENOME_TAGS)
ratings = pd.read_csv(PATH_RATING)

#Разделение на тестовую и обучающую выборку

In [ ]:
users=list(ratings['userId'].unique())
movies=list(ratings['movieId'].unique())

split = np.random.rand(len(ratings)) > train_test_split
train = ratings[split]
test = ratings[~split]

In [ ]:
print(train.shape , test.shape)

(16001289, 4) (3998974, 4)


#SVD-like алгоритм


In [ ]:
ratings.userId = ratings.userId.astype('category').cat.codes.values
ratings.movieId = ratings.movieId.astype('category').cat.codes.values

In [ ]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}
ratings['userId'] = ratings['userId'].apply(lambda x: userid2idx[x])
ratings['movieId'] = ratings['movieId'].apply(lambda x: movieid2idx[x])


In [ ]:
index=list(ratings['userId'].unique())
columns=list(ratings['movieId'].unique())
index=sorted(index)
columns=sorted(columns)
n_movies=len(index)
n_users=len(columns)
k = 64

In [ ]:
user_input = Input(shape=(1,),name='user_input',dtype='int64')
user_embedding = Embedding(n_users, k, name='user_embedding')(user_input)
user_vec = Flatten(name='FlattenUsers')(user_embedding)

In [ ]:
movie_input=Input(shape=(1,),name='movie_input',dtype='int64')
movie_embedding=Embedding(n_movies,k,name='movie_embedding')(movie_input)
movie_vec=Flatten(name='FlattenMovies')(movie_embedding)

In [ ]:
sim=dot([user_vec,movie_vec],name='Simalarity-Dot-Product',axes=1)
model =keras.models.Model([user_input, movie_input],sim)

In [ ]:
model.compile(optimizer=Adam(lr=1e-4),loss='mse')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
print(train.shape)
batch_size=62
epochs=10

(16001289, 4)


In [ ]:
History = model.fit([train.userId,train.movieId],train.rating, batch_size=batch_size,epochs =epochs, validation_data = ([test.userId,test.movieId],test.rating),verbose = 1)

Epoch 1/5


InvalidArgumentError: ignored

# LightFM


In [ ]:
data = fetch_movielens(min_rating=4.0)

#print training and testing data
print(repr(data['train']))
print(repr(data['test']))

model = LightFM(loss='warp')
model.fit(data['train'], epochs=10, num_threads=2)